# Setup

In [4]:
from pathlib import Path
import numpy as np
import gensim

base_path = Path("go-full")

In [5]:
wv = gensim.models.KeyedVectors.load(str(base_path / "owl2vec" / "ontology.embeddings"), mmap="r")

[11/17/24 22:39:43] INFO     loading KeyedVectors object from go-full/owl2vec/ontology.embeddings      ]8;id=405766;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=460514;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#481\481]8;;\

                    INFO     loading wv recursively from go-full/owl2vec/ontology.embeddings.wv.* with ]8;id=457919;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=876201;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#515\515]8;;\
                             mmap=r                                                                                

                    INFO     loading vectors from go-full/owl2vec/ontology.embeddings.wv.vectors.npy   ]8;id=84441;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=365654;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#520\520]8;;\
                             with mmap=r                                                                           

                    INFO     loading syn1neg from go-full/owl2vec/ontology.embeddings.syn1neg.npy with ]8;id=660783;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=469933;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#520\520]8;;\
                             mmap=r                                                                                

                    INFO     setting ignored attribute cum_table to None                               ]8;id=131835;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=227587;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#553\553]8;;\

[11/17/24 22:39:46] INFO     Word2Vec lifecycle event {'fname': 'go-full/owl2vec/ontology.embeddings', ]8;id=676627;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=347841;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#447\447]8;;\
                             'datetime': '2024-11-17T22:39:46.603187', 'gensim': '4.3.3', 'python':                
                             '3.8.20 | packaged by conda-forge | (default, Sep 30 2024, 17:48:42)                  
                             \n[Clang 17.0.6 ]', 'platform': 'macOS-15.1-arm64-arm-64bit', 'event':                
                             'loaded'}                                                                             

In [3]:
import logging
from rich.logging import RichHandler

logging.basicConfig(level="INFO", format="%(message)s", handlers=[RichHandler()])

from pathlib import Path
import numpy as np
import gensim
from nltk import word_tokenize
import re

# Load the model
wv = gensim.models.KeyedVectors.load(str(base_path / "owl2vec" / "ontology.embeddings"), mmap="r")


# Helper functions from OWL2Vec_Plus.py
def pre_process_words(words):
    text = ' '.join([re.sub(r'https?:\/\/.*[\r\n]*', '', word, flags=re.MULTILINE) for word in words])
    tokens = word_tokenize(text)
    processed_tokens = [token.lower() for token in tokens if token.isalpha()]
    return processed_tokens


# Load classes and annotations
classes = [line.strip() for line in open(base_path / "split" / "classes.txt").readlines()]
uri_label = {}

label_words = 0
available_words = 0
preproc_words = 0
multilabels = []
for line in open(base_path / "cache" / "annotations.txt").readlines():
    tmp = line.strip().split()
    if tmp[1] == "preferred_label" or tmp[1] == "http://www.w3.org/2000/01/rdf-schema#label":
        uri_label[tmp[0]] = pre_process_words(tmp[2:])

        if tmp[0] in uri_label and tmp[0] not in multilabels:
            multilabels.append(tmp[0])
        else:
            preproc_words += len(uri_label[tmp[0]])
        label_words += len(tmp) - 2
        available_words += len(tmp) - 2
    else:
        available_words += len(tmp) - 2

logging.info(f"Average number of label words available per class: {label_words / len(classes)}")
logging.info(f"Average number of words available per class: {available_words / len(classes)}")


n_s = []
lost_words = 0
def embed(model, instances, uri_label: dict):
    def word_embeding(inst):
        global lost_words
        v = np.zeros(model.vector_size)
        if inst in uri_label:
            words = uri_label.get(inst)
            n = 0
            for word in words:
                if word in model.wv.index_to_key:
                    v += model.wv.get_vector(word)
                    n += 1
                else:
                    lost_words += 1
            return (v / n if n > 0 else v, n)
        else:
            return (v, 0)

    feature_vectors = []
    for instance in instances:
        # Get both URI and word embeddings
        # v_uri = model.get_vector(instance) if instance in model.index_to_key else np.zeros(model.vector_size)
        v_word, n = word_embeding(inst=instance)
        n_s.append(n)
        # Concatenate them
        feature_vectors.append(v_word)

    return feature_vectors


# Calculate embeddings
classes_e = embed(model=wv, instances=classes, uri_label=uri_label)
logging.info(f"Number of words lost: {lost_words}")
logging.info(f"Average number of words used per class: {np.mean(n_s)}")
# Extract GO IDs from class URIs and create dictionary
go_ids = [cls.split('/')[-1] for cls in classes]
embeddings_dict = {go_id: emb for go_id, emb in zip(go_ids, classes_e)}

# Save embeddings
np.save(str(base_path / "owl2vec" / "ontology.embeddings.npy"), embeddings_dict)

print(f"Saved embeddings for {len(embeddings_dict)} classes")
print(f"Each embedding has dimension: {len(next(iter(embeddings_dict.values())))} (word embeddings)")

[11/17/24 22:12:05] INFO     loading KeyedVectors object from go-basic/owl2vec/ontology.embeddings     ]8;id=801713;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=792378;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#481\481]8;;\

                    INFO     loading wv recursively from go-basic/owl2vec/ontology.embeddings.wv.*     ]8;id=781941;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=588560;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#515\515]8;;\
                             with mmap=r                                                                           

                    INFO     loading vectors from go-basic/owl2vec/ontology.embeddings.wv.vectors.npy  ]8;id=898516;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=464725;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#520\520]8;;\
                             with mmap=r                                                                           

                    INFO     loading syn1neg from go-basic/owl2vec/ontology.embeddings.syn1neg.npy     ]8;id=393618;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=338687;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#520\520]8;;\
                             with mmap=r                                                                           

                    INFO     setting ignored attribute cum_table to None                               ]8;id=583454;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py\utils.py]8;;\:]8;id=738678;file:///Users/mmtf/miniforge3/envs/owl2vec/lib/python3.8/site-packages/gensim/utils.py#553\553]8;;\

...

[11/17/24 22:12:38] INFO     Average number of label words available per class: 5.265237633365665   ]8;id=242804;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_35860/723607119.py\723607119.py]8;;\:]8;id=836759;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_35860/723607119.py#46\46]8;;\

                    INFO     Average number of words available per class: 38.02019398642095         ]8;id=258196;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_35860/723607119.py\723607119.py]8;;\:]8;id=329814;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_35860/723607119.py#47\47]8;;\

[11/17/24 22:12:42] INFO     Number of words lost: 0                                                ]8;id=633047;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_35860/723607119.py\723607119.py]8;;\:]8;id=669288;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_35860/723607119.py#83\83]8;;\

                    INFO     Average number of words used per class: 4.110164888457808              ]8;id=140576;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_35860/723607119.py\723607119.py]8;;\:]8;id=605438;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_35860/723607119.py#84\84]8;;\

Saved embeddings for 51550 classes
Each embedding has dimension: 100 (word embeddings)


In [4]:
duplicates = []
for line in open(base_path / "cache" / "annotations.txt").readlines():
    tmp = line.strip().split()
    if tmp[0] in multilabels:
        duplicates.append(tmp)
logging.info(f"Number of duplicate classes: {len(duplicates)}")


[11/15/24 18:46:20] INFO     Number of duplicate classes: 452011                                     ]8;id=279699;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_85971/548770051.py\548770051.py]8;;\:]8;id=413519;file:///var/folders/pb/_0h8x45x091307x0pwzykw8h0000gn/T/ipykernel_85971/548770051.py#6\6]8;;\

In [11]:
len(duplicates)

NameError: name 'duplicat' is not defined

In [6]:
preproc_words / len(classes)

0.5823162462706808